In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')


if os.path.exists("/content/drive/MyDrive/ai_kepek") == False:
    !mkdir /content/drive/MyDrive/ai_kepek

#@markdown A neved az ugyanaz mint a képeknél amit feltöltöttél

# ahogy elnevezted a képeket, a zárojel előtt
neved='' #@param{type: 'string'}

#@markdown A model neve az amit a hugging facen van a profilodban
# ami a hugingfacen van usernev/modelnev
modeled_neve = '' #@param{type: 'string'}

!nvidia-smi
!pip install diffusers
!pip install transformers scipy ftfy accelerate

import random
import string


from PIL import Image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


from diffusers import DiffusionPipeline
import torch
pipeline = DiffusionPipeline.from_pretrained( modeled_neve, torch_dtype=torch.float16)

pipeline.to("cuda")



import pandas as pd
import random

df = pd.read_csv('https://raw.githubusercontent.com/misrori/ai/main/styles.tsv', sep='\t')
df = df.loc[df['style'].str.contains('Portrait', case=False)].reset_index(drop=True)
additional_prompt_params = [
    'Fantasy art',
    'Cinematic atmosphere',
    'Photorealistic',
    'Highly detailed',
    'Dynamic lighting',
    'Sci-fi elements',
    'Mystical ambiance',
    'Vibrant colors',
    'Subtle textures',
    'Futuristic design',
    'Historical references',
    'Emotional expressions',
    'Minimalistic approach',
    'Abstract concepts',
    'Whimsical elements',
    'Nature-inspired',
    'Symmetry and balance',
    'Contrast in composition',
    'Play with shadows',
    'Unique perspectives',
    'Monochromatic scheme',
    'Intricate patterns',
    'Surrealistic elements',
    'Interactive elements',
    'Ambient soundscapes',
    'Detailed backgrounds',
    'Character interactions',
    'Use of negative space',
    'Geometric shapes',
    'Impressionistic style',
    'Weather influences',
    'Varying focal points',
    'Hand-drawn elements',
    'Collage-style composition',
    'Texture overlays',
    'Whimsical characters',
    'Dynamic poses',
    'Use of symbolism',
    'Capturing motion',
    'Vintage aesthetics',
    'Bold typography',
    'Comic book style',
    'Layered storytelling',
    'Interactive environments',
    'Emphasis on emotions',
    'Abstract symbolism',
    'Innovative perspective',
    'Harmony in color palette',
    'Character backstories',
    'Ambient lighting',
    'Blend of reality and fantasy',
    'Experimental techniques',
    'Dramatic angles',
    'Architectural details',
    'Exploration of cultural themes',
    'Steampunk elements',
    'Emphasis on contrast',
    'Interactive narratives',
    'Mix of genres',
    'Magical realism',
    'Unexpected twists',
    'Reflections and mirrors',
    'Blend of traditional and digital',
    'Hidden details',
    'Use of symbolism',
    'Dreamlike sequences',
    'Dynamic compositions',
    'Play with scale',
    'Visual metaphors',
    'Diverse character representation',
    'Story within a story',
    'Intriguing facial expressions',
    'Unconventional color choices',
    'Symbolic objects',
    'Weather effects',
    'Mysterious atmospheres',
    'Creative use of negative space',
    'Subtle storytelling cues',
    'Abstract shapes and forms',
    'Blurring of reality and fiction',
    'Interactive elements',
    'Time-travel concepts',
    'Use of reflections',
    'Clever use of shadows',
    'Unexpected perspectives',
    'Retro-futuristic vibes',
    'Blend of traditional and digital',
    'Unconventional materials',
    'Immersive environments',
    'Symbolic use of color',
    'Fluid transitions',
    'Hidden Easter eggs',
    'Clever use of symbolism',
    'Interactive user experiences',
    'Narrative-driven compositions',
    'Unique framing techniques',
    'Blend of realism and abstraction'
]


df['promt'] = [f"Square photo of {neved} person style of {x}, inspired by {x} 4k, digital art, {', '.join(random.choices(additional_prompt_params, k=2))}" for x in df['nev']]
df['artist'] = df['nev'].str.replace(', ', '_').str.lower().str.replace(' ', '').str.replace('(', '_').str.replace(')', '_')

kep_id = 1
for index, row in df.iterrows():

    imgs = pipeline(row['promt'], height = 560, width=560, num_images_per_prompt = 2 ).images

    for img in imgs:
      img.save(f"/content/drive/MyDrive/ai_kepek/{row['artist']}_{kep_id}.png")
      kep_id+=1


os.system(f"ffmpeg -framerate 10 -pattern_type glob -i '/content/drive/MyDrive/ai_kepek/*.png' -c:v libx264 -pix_fmt yuv420p /content/drive/MyDrive/ai_kepek/video.mp4")


# Egyéni promt megadása

In [ ]:


promt_style = "style of Tristan Eaton, 4k, digital art"  #@param{type: 'string'}
number_of_images = 4  #@param{type: 'number'}
imgs = pipeline(f"Square photo of {neved} person {promt_style}", height = 560, width=560, num_images_per_prompt = number_of_images, negative_promt = '' ).images

grid = image_grid(imgs, rows=len(imgs), cols=1)
grid